# Deutch-Jozsa Algorithm
DJ is a quantum algorithm that exhibits an exponential speedup over classical implementation. Given n-qubits for n-number of parameters, its time complexity is O(1) compared to the worst case on classical implementation which is O(2^(n-1) + 1). Though it has no practical application, it clearly illustrates the quantum mechanical effects of superposition, interference and entanglement. 

In [1]:
// NOTE: Namespace not accepted due to the non-contiguous nature of cells
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert as Convert;
open Microsoft.Quantum.Measurement; 
open Microsoft.Quantum.Arrays;

## 1-Qubit Oracles
Oracles are the functions which property we want to determine. Note the use of [ancilla](https://en.wikipedia.org/wiki/Ancilla_bit) qubit to respresent the result. This is a requirement in [reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) like QC.

In [2]:
operation Const0(qubits: Qubit[]) : Unit {
    // This is universal and can be used by any
    // n-qubit oracle. It's equal to doing nothing.
    ApplyToEach(I, qubits); 
}

operation Const1(qubits: Qubit[]) : Unit {       
    let indexOfAncilla = Length(qubits)-1;
    X(qubits[indexOfAncilla]);      
}

operation Balanced0(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
}

operation Balanced1(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
    X(qubits[1]);
}

Const0, Const1, Balanced0, Balanced1

## 2-Qubit Oracles

In [3]:
operation Balanced_2Qubit0(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[2]);
}

operation Balanced_2Qubit1(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[2]);
    X(qubits[2]);
}

operation Balanced_2Qubit2(qubits: Qubit[]) : Unit {
    CNOT(qubits[1], qubits[2]);        
}

operation Balanced_2Qubit3(qubits: Qubit[]) : Unit {
    CNOT(qubits[1], qubits[2]);  
    X(qubits[2]);
}

operation Balanced_2Qubit4(qubits: Qubit[]) : Unit {
    CNOT(qubits[0],qubits[2]);
    CNOT(qubits[1],qubits[2]);
}

operation Balanced_2Qubit5(qubits: Qubit[]) : Unit {
    CNOT(qubits[0],qubits[2]);
    CNOT(qubits[1],qubits[2]);
    X(qubits[2]);
}

Balanced_2Qubit0, Balanced_2Qubit1, Balanced_2Qubit2, Balanced_2Qubit3, Balanced_2Qubit4, Balanced_2Qubit5

## Circuit
To accomodate different oracles, we isolate the circuit creation in its own operation. Note that Q# has no support on named delegate, only lambda.

In [24]:
operation DeutchJozsa(bits: Int, oracle: ((Qubit[]) => Unit)) : Result {
    use qubits = Qubit[bits + 1] {

        X(qubits[bits]);

        ApplyToEach(H,qubits);

        oracle(qubits);

        ApplyToEach(H,qubits);            

        mutable result = Zero;   
        // Hardcoded logic to cater only for 2-qubit oracle      
        if  bits == 2 {       
            ApplyToEach(X, [qubits[0], qubits[1]]);
            CCNOT(qubits[0], qubits[1], qubits[2]);               
        }

        set result = M(qubits[bits]);

        ResetAll(qubits);

        return result;
    }   
}

DeutchJozsa

## Trying out Types

In [5]:
// NOT WORKING! Bug?
//  https://docs.microsoft.com/en-us/azure/quantum/user-guide/language/expressions/itemaccessexpressions#item-access-for-user-defined-types

newtype DJParams = (
                        QBitCount : Int,
                        Oracles   :((Qubit[]) => Unit)[] // Array of void delegates accepting Qubit array as param                                                         
                    );

newtype Complex = (Real: Double, Imaginary : Double);
//let complex = Complex(1.,0.);

DJParams, Complex

In [33]:
@EntryPoint()
operation DJMain() : Result[] {

        // 1-qubit oracles        
        let oracles1Qubit = [Const0, Const1, Balanced0, Balanced1];
        
        // 2-qubit oracles      
        // CAUTION! This causes error in IonQ when submitting all items. 
        // From tests, the max number of items accepted is 3. BUG?
        let oracles2Qubit = [
                            //Const0
                            //, Const1
                              Balanced_2Qubit0
                            , Balanced_2Qubit1
                            , Balanced_2Qubit2];
                        //, Balanced_2Qubit3
                        //, Balanced_2Qubit4
                        //, Balanced_2Qubit5];

        // 1-qubit params
        //let oracles = oracles1Qubit;
        //let qbitCount = 1;

        // 2-Qubit params
        let oracles = oracles2Qubit;
        let qbitCount = 2;

        mutable results = ConstantArray(Length(oracles),Zero);
        
        for i in 0 .. Length(oracles)-1 {            
            set results w/= i <- DeutchJozsa(qbitCount,oracles[i]);          
        }               

        return results;
}

DJMain

In [8]:
%simulate DJMain

One, One, One

In [9]:
%estimate DJMain

Metric,Sum,Max
CNOT,31,31
QubitClifford,34,34
R,0,0
Measure,12,12
T,21,21
Depth,15,15
Width,3,3
QubitCount,3,3
BorrowedWidth,0,0


In [10]:
%azure.connect "/subscriptions/a6c78a63-6e6d-4d66-8d84-3989715f3111/resourceGroups/quantum/providers/Microsoft.Quantum/Workspaces/demo-ws1"

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EMBA7794R to authenticate.

Connected to Azure Quantum workspace demo-ws1 in location westus.


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,82
ionq.simulator,Available,0
honeywell.hqs-lt-1.0,Unavailable,0
honeywell.hqs-lt-1.0-apival,Available,0


In [11]:
%azure.target ionq.qpu

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,82


In [34]:
%azure.submit DJMain jobName="DJ 3 Qubits, 3 const oracles" shots="1024"

Submitting DJMain to target ionq.qpu...
Job successfully submitted for 1024 shots.
   Job name: DJ 3 Qubits, 3 const oracles
   Job ID: 781f98bc-0a84-41a0-8569-ba160ae0c68c


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
"DJ 3 Qubits, 3 const oracles",781f98bc-0a84-41a0-8569-ba160ae0c68c,Waiting,ionq.qpu,03/29/2021 19:25:24,,


In [27]:
%azure.status "457c620e-d4d2-40ae-9231-d8d9498f63f9"

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
"DJ 3 Qubits, 2 const oracles",457c620e-d4d2-40ae-9231-d8d9498f63f9,Succeeded,ionq.qpu,03/29/2021 18:58:53,03/29/2021 19:00:38,03/29/2021 19:00:48


In [31]:
%azure.output "6b025829-8b38-4026-ba30-c1320c12b7b0"

Result,Frequency,Histogram
"[0,0,0]",0.1264,
"[1,0,0]",0.0148,
"[0,1,0]",0.03250000000000001,
"[1,1,0]",0.006,
"[0,0,1]",0.5805000000000001,
"[1,0,1]",0.053000000000000005,
"[1,1,1]",0.0235,
"[0,1,1]",0.1652,
